In [44]:
from chembl_webresource_client.new_client import new_client
import json
import numpy as np
import pandas as pd
import requests

# Task 1

In [2]:
molecule = new_client.molecule
approved_drugs = molecule.filter(max_phase=4).order_by('first_approval', 'pref_name')
len(approved_drugs)

4194

# Task 2

In [3]:
drug_list = list()

for drug in approved_drugs.filter(first_approval__gte=2013):
    drug_list.append((drug['pref_name'], drug['molecule_chembl_id'], drug['first_approval']))

619

In [25]:
drug_id  = [d[1] for d in drug_list]
look_up = dict()
for d in drug_id:
    look_up[d] = set()

In [26]:

activity = new_client.activity.filter(molecule_chembl_id__in=drug_id).only(['molecule_chembl_id', 'target_chembl_id'])
step = 100
for i in range(0, len(activity), step):
    for a in activity[i:i+step]:
        look_up[a['molecule_chembl_id']].add(a['target_chembl_id'])



In [54]:
target_dict = dict()
for key in look_up:
    val = list(look_up[key])
    if len(val) != 0:
        tar = new_client.target.filter(target_chembl_id__in=val).only(['target_components'])
        if len(set(c['accession'] for t in tar for c in t['target_components'])) != 0:
            target_dict[val[0]] = set(c['accession'] for t in tar for c in t['target_components'])

med_list = list()
for key in target_dict:
    med_list.append(len(target_dict[key]))
    
print(np.median(med_list))

4.0


# Task 3

In [76]:

url = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=100&accession="



keywords = dict()
for key in target_dict:
    for up_id in target_dict[key]:
        if isinstance(up_id, str):
            request = requests.get(url + up_id, headers={ "Accept" : "application/json"})
        
        if not request.ok:
            request.raise_for_status()
        
        t = json.loads(request.text)
        for words in t:
            if 'keywords' in words.keys():
                for word in words['keywords']:
                    if word['value'] in keywords:
                        keywords[word['value']] += 1
                    else:
                        keywords[word['value']] = 1


In [82]:
pd.DataFrame.from_dict(sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10])

,0,1
0,Reference proteome,1424
1,3D-structure,1133
2,Phosphoprotein,1065
3,Nucleotide-binding,912
4,ATP-binding,859
5,Alternative splicing,837
6,Transferase,825
7,Membrane,778
8,Kinase,758
9,Cytoplasm,709


In [84]:
sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]

[('Reference proteome', 1424),
 ('3D-structure', 1133),
 ('Phosphoprotein', 1065),
 ('Nucleotide-binding', 912),
 ('ATP-binding', 859),
 ('Alternative splicing', 837),
 ('Transferase', 825),
 ('Membrane', 778),
 ('Kinase', 758),
 ('Cytoplasm', 709)]